In [5]:
from keras.models import Sequential
from keras.layers import Dense

import numpy as np
# fix random seed for reproducibility
np.random.seed(42)

import pandas as pd

In [6]:
#Folder for the dataset
datasetFolder = '~/dbpedia2016/dataset/'

#Number of files
numberOfFiles = 256

#Test split
testSplit=0.25

In [7]:
def load_data(datasetFolder, datasetXFile, datasetYFile):
    print('Loading {} & {}'.format(datasetXFile, datasetYFile))
    # load file
    dataX=pd.read_csv(datasetFolder + datasetXFile, engine='c', na_filter=False)
    dataY=pd.read_csv(datasetFolder + datasetYFile, engine='c', na_filter=False)

    # delete the id fields
    del dataX['id']
    del dataY['id']

    return dataX, dataY

In [8]:
dataX, dataY = load_data(datasetFolder,'datasetX_1.csv', 'datasetY_1.csv')

Loading datasetX_1.csv & datasetY_1.csv


In [9]:
dataX.head()

,walk_1,walk_2,walk_3,walk_4,walk_5,walk_6,walk_7,walk_8,walk_9,walk_10,...,walk_1096,walk_1097,walk_1098,walk_1099,walk_1100,walk_1101,walk_1102,walk_1103,walk_1104,walk_1105
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
dataY.head()

,class_1,class_2,class_3,class_4,class_5,class_6,class_7,class_8,class_9,class_10,...,class_530,class_531,class_532,class_533,class_534,class_535,class_536,class_537,class_538,class_539
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
print("Input Features: {}".format(dataX.shape[1]))
print("Output Classes: {}".format(dataY.shape[1]))

Input Features: 1105
Output Classes: 539


In [12]:
# create model
logisticRegression = Sequential(name='Simple Logistic Regression')
logisticRegression.add(Dense(dataY.shape[1], input_dim=dataX.shape[1], activation='sigmoid'))

simpleModel = Sequential(name='2 Fully Connected Layers')
simpleModel.add(Dense(1024, input_dim=dataX.shape[1], activation='relu'))
simpleModel.add(Dense(dataY.shape[1], activation='sigmoid'))

deepModel = Sequential(name='Deep Model (5 Dense Layers)')
deepModel.add(Dense(2048, input_dim=dataX.shape[1], activation='relu'))
deepModel.add(Dense(1024, activation='relu'))
deepModel.add(Dense(768, activation='relu'))
deepModel.add(Dense(512, activation='relu'))
deepModel.add(Dense(dataY.shape[1], activation='sigmoid'))

models = [logisticRegression, simpleModel, deepModel]

In [13]:
# Compile model
import keras.backend as K

def count_predictions(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives, predicted_positives, possible_positives

def f1score(y_true, y_pred):
    true_positives, predicted_positives, possible_positives = count_predictions(y_true, y_pred)
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1score = 2.0 * precision * recall / (precision+recall+ K.epsilon())
    return f1score

for model in models:
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1score])

In [14]:
def fit_data(model, dataX, dataY):
    # Fit the model
    #model.fit(dataX.as_matrix(), dataY.as_matrix(), nb_epoch=5, verbose=2, batch_size=256)
    return model.train_on_batch(dataX.as_matrix(), dataY.as_matrix())

In [15]:
def countPredictions(y_true, y_pred):
    true_positives = np.sum(np.round(y_pred*y_true))
    predicted_positives = np.sum(np.round(y_pred))
    possible_positives = np.sum(y_true)
    return true_positives, predicted_positives, possible_positives

In [16]:
#Randomize the list of numbers so we can split train and test dataset
listOfFiles=list(range(1,numberOfFiles+1))
import random
random.shuffle(listOfFiles)
splitIndex=int((1-testSplit)*numberOfFiles)

numberOfEons = 5
for eon in range(0, numberOfEons):
    print('\n***********************************\nEon {}/{}'.format(eon+1, numberOfEons))
    for trainIndex in range(0,splitIndex):
        print('==========================================')
        print('Learning for file {} / {} : datasetX/Y_{}'.format(trainIndex+1, splitIndex, listOfFiles[trainIndex]))
        dataX, dataY = load_data(datasetFolder,'datasetX_{}.csv'.format(listOfFiles[trainIndex]), 'datasetY_{}.csv'.format(listOfFiles[trainIndex]))
        print('Training...')
        for model in models:
            #print('Model = {}'.format(model.name))
            loss, f1score=fit_data(model,dataX, dataY)
            print('   Model = {} \t loss = {:.4f} \t f1-score = {:.4f}'.format(model.name, loss, f1score))
            
        
    counts = {} 
    for model in models:
        counts[model.name] = {'true_positives':0, 'predicted_positives':0, 'possible_positives':0}
    
    print('==========================================\n')
    for testIndex in range(splitIndex, numberOfFiles):
        dataX, dataY = load_data(datasetFolder,'datasetX_{}.csv'.format(listOfFiles[testIndex]), 'datasetY_{}.csv'.format(listOfFiles[testIndex]))
        print ('Testing for file {} / {} : datasetX/Y_{}'.format(testIndex+1, numberOfFiles, listOfFiles[trainIndex]))
        for model in models:
            predY=model.predict_on_batch(dataX.as_matrix())
            true_positives, predicted_positives, possible_positives = countPredictions(dataY.as_matrix(), predY)
            counts[model.name]['true_positives'] += true_positives
            counts[model.name]['predicted_positives'] += predicted_positives
            counts[model.name]['possible_positives'] += possible_positives
    print('==========================================\n')
    
    
    print('{}. Eon {}/{} - Testing score:'.format(eon+1, eon+1, numberOfEons))
    for model in models:
        count = counts[model.name]
        precision = (count['true_positives'])/(count['predicted_positives']+0.0001)
        recall = (count['true_positives'])/(count['possible_positives']+0.0001)
        f1score = 2.0 * precision * recall / (precision+recall+0.0001)
        print(' - Model = {} \t f1-score = {:.4f}\t precision = {:.4f} \t recall = {:.4f}'.format(model.name, f1score, precision, recall))
    print('==========================================\n')


***********************************
Eon 1/5
Learning for file 1 / 192 : datasetX/Y_213
Loading datasetX_213.csv & datasetY_213.csv
Training...
   Model = Simple Logistic Regression 	 loss = 0.6938 	 f1-score = 0.0099
   Model = 2 Fully Connected Layers 	 loss = 0.6927 	 f1-score = 0.0096
   Model = Deep Model (5 Dense Layers) 	 loss = 0.6934 	 f1-score = 0.0123
Learning for file 2 / 192 : datasetX/Y_135
Loading datasetX_135.csv & datasetY_135.csv
Training...
   Model = Simple Logistic Regression 	 loss = 0.6931 	 f1-score = 0.0101
   Model = 2 Fully Connected Layers 	 loss = 0.6870 	 f1-score = 0.0082
   Model = Deep Model (5 Dense Layers) 	 loss = 0.6891 	 f1-score = 0.0125
Learning for file 3 / 192 : datasetX/Y_222
Loading datasetX_222.csv & datasetY_222.csv
Training...
   Model = Simple Logistic Regression 	 loss = 0.6922 	 f1-score = 0.0098
   Model = 2 Fully Connected Layers 	 loss = 0.6804 	 f1-score = 0.0070
   Model = Deep Model (5 Dense Layers) 	 loss = 0.6788 	 f1-score = 0.

==================================================
# Incoming Relationship Presence
==================================================
1. Eon 1/5 - Testing score: 
 - Model = Simple Logistic Regression 	 f1-score = 0.1055	 precision = 0.8051 	 recall = 0.0565
 - Model = 2 Fully Connected Layers 	 f1-score = 0.0239	 precision = 0.8424 	 recall = 0.0121
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.1171	 precision = 0.7318 	 recall = 0.0637
2. Eon 2/5 - Testing score: 
 - Model = Simple Logistic Regression 	 f1-score = 0.1303	 precision = 0.8122 	 recall = 0.0709
 - Model = 2 Fully Connected Layers 	 f1-score = 0.1382	 precision = 0.7603 	 recall = 0.0760
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.1804	 precision = 0.7729 	 recall = 0.1021
3. Eon 3/5 - Testing score:
 - Model = Simple Logistic Regression 	 f1-score = 0.1456	 precision = 0.8061 	 recall = 0.0800
 - Model = 2 Fully Connected Layers 	 f1-score = 0.1705	 precision = 0.7771 	 recall = 0.0958
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.2005	 precision = 0.7814 	 recall = 0.1150
4. Eon 4/5 - Testing score:
 - Model = Simple Logistic Regression 	 f1-score = 0.1546	 precision = 0.8073 	 recall = 0.0855
 - Model = 2 Fully Connected Layers 	 f1-score = 0.1815	 precision = 0.7781 	 recall = 0.1028
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.2009	 precision = 0.7851 	 recall = 0.1152
5. Eon 5/5 - Testing score:
 - Model = Simple Logistic Regression 	 f1-score = 0.1656	 precision = 0.8000 	 recall = 0.0924
 - Model = 2 Fully Connected Layers 	 f1-score = 0.1885	 precision = 0.7805 	 recall = 0.1072
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.2044	 precision = 0.7809 	 recall = 0.1176